In [ ]:
import pandas as pd
import geopandas as gp 
import numpy as np 
from pandas import read_csv
import requests

## VEST documentation

Louisiana
---------
Election results from LA Secretary of State (https://voterportal.sos.la.gov/static/)
Precinct shapefile from LA House of Representatives (http://house.louisiana.gov/H_Redistricting2011/default_LouisianaPrecinctShapefiles)

Early votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Election results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne

The following precincts were modified to reflect alterations enacted prior to the 2018 election:

Avoyelles: Merge 2-5B/6-1A  
Plaquemines: Merge 2-1/2-2, 4-1/4-2, 5-1/5-2  
St. Charles: Merge 2-6/2-7, 3-1/3-6, 3-3/3-4, 6-2/6-3, 6-4/6-5  
Vermilion: Split 49B-1/49B-2  
West Baton Rouge: Split 2-A/2-B; 11-A/11-B  
  
G18SOSRARD - Kyle Ardoin (Republican Party)  
G18SOSREDM - "Rick" Edmongs (Republican Party)  
G18SOSRSTO - Julie Stokes (Republican Party)  
G18SOSRKEN - Thomas J. Kennedy III (Republican Party)  
G18SOSRCRO - A.G. Crowe (Republican Party)  
G18SOSRCLO - Heather Cloud (Republican Party)  
G18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)  
G18SOSDFRE - Renee Fontenot Free (Democratic Party)  
G18SOSNMOR - Matthew Paul "Matt" Moreau (No Party)  
  
R18SOSRARD - Kyle Ardoin (Republican Party)  
R18SOSDCOL - "Gwen" Collins-Greenup (Democratic Party)  

## Links for data

Election data: https://voterportal.sos.la.gov/graphical

Precinct shapefile from LA House of Representatives: https://house.louisiana.gov/H_redistricting2011/Shapefiles/2018%20LA%20Precincts%20for%20the%20Web.zip

## Load datasets

In [ ]:
#Secretary of State General Election Run-off 
SoSG = pd.read_csv('./raw-data/election_results_la/SoSGeneral2018_LAPrecincts.csv')

#Secretary of State Jungle Primary
SoSP = pd.read_csv('./raw-data/election_results_la/SoSPrimary2018_LAPrecincts.csv')

#Load precinct shapefile
precincts = gp.read_file('./raw-data/2018 LA Precincts for the Web/2018 LA Precincts for the Web.shp')

#Load final VEST file
final = gp.read_file('./final_vest_la_2018/la_2018.shp')

#Set options for pandas displays
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
print(final[final["COUNTYFP10"]=="079"])

## Modify election data (alpha parishes and early vote)

### View VEST's early vote processing documentation

Early votes were reported at the parish level. These were distributed by candidate to precincts based on their share of the precinct-level reported vote.

Election results from the following parishes include "alpha" precincts in which voters within the same geographic boundaries are assigned to separate precincts by the first letter of their surname: Ascension, Assumption, Bossier, Caddo, East Baton Rouge, Lafayette, Lafourche, Rapides, St. Charles, St. Landry, Terrebonne

### View the election data columns

In [ ]:
print(SoSG.columns)
print(SoSP.columns)

### View the final VEST  columns

In [ ]:
print(final.columns)

### Rename the election dataset fields to match those in the VEST file

In [ ]:
#Create a rename dictionary for SoSP file based on the corresponding fields in the VEST file
SoSP_rename_dict = {'Kyle Ardoin (REP)':'G18SOSRARD','Heather Cloud (REP)':'G18SOSRCLO','"Gwen" Collins-Greenup (DEM)':'G18SOSDCOL',
            'A.G. Crowe (REP)':'G18SOSRCRO','"Rick" Edmonds (REP)':'G18SOSREDM', 'Renee Fontenot Free (DEM)':'G18SOSDFRE',
            'Thomas J. Kennedy III (REP)':'G18SOSRKEN', 'Matthew Paul "Matt" Moreau (NOPTY)':'G18SOSNMOR','Julie Stokes (REP)':'G18SOSRSTO'}

#Rename the SoSP file
SoSP.rename(columns = SoSP_rename_dict, inplace=True)

#Rename the two election result fields in the SoSG file based on their name in the VEST file
SoSG.rename(columns = {'Kyle Ardoin (REP)':'R18SOSRARD','"Gwen" Collins-Greenup (DEM)':'R18SOSDCOL'},inplace=True)

### View the the column names of the election results and the final VEST file

In [ ]:
print('SoSG columns: ', SoSG.columns)
print('SoSP columns: ', SoSP.columns)
print('VEST columns: ', final.columns)

### Create a function to retrieve the totals of all of the columns

In [ ]:
'''This function returns a dictionary of election results column names and totals for a dataframe.
Input: pandas dataframe
Output: dictionary with column name as key and the total sum of the column as the value'''

def get_column_totals(df):
    column_totals = [] #list for column sums 
    column_name = [] #list for column names
    for i in df.columns:
        if '18' in i: #only election results in the df
            df[i] = df[i].astype(float) #converts the column to float
            column_name.append(i) #add the column name to list of column names
            col_tot = df[i].sum() #gets the sume of the column
            col_tot = round(col_tot,0) #rounds the column total to nearest whole number
            column_totals.append(col_tot) #adds the column total to the list of column sums
    tot_dict = dict(zip(column_name,column_totals)) #zips the two lists into a dictionary
    return tot_dict #returns the dictionary

### Compare the total votes for each candidate in the raw data to the VEST file

In [ ]:
SoSP_tot = get_column_totals(SoSP)
SoSG_tot = get_column_totals(SoSG)
final_tot = get_column_totals(final)
SoSP_tot.update(SoSG_tot) #appends the SoSG_tot to the SoSP_tot dict (as this is the order that they are in the VEST file)
SoS_tot = SoSP_tot
SoSP_tot = get_column_totals(SoSP) #reruns SoSP_tot for SosP
print('SoSP dictionary: ', SoSP_tot)
print('\nSoSG dictionary: ', SoSG_tot)
print('\nSoS total dictionary: ', SoS_tot)
print('\nVEST dictionary: ', final_tot)
matching = SoS_tot == final_tot #boolean result between raw data and final election results 
print('\nDo the election totals between the raw data and the final VEST file match (T/F)? ', str(matching))

### Now that we know that the results are the same, we need to assign the early vote by parish to the individual precincts based on candidate performance. 

In [ ]:
#List of parishes with 'alpha' precincts from VEST documentation above
alpha_co = ['Ascension', 'Assumption', 'Bossier', 'Caddo', 'East Baton Rouge', 'Lafayette', 'Lafourche', 'Rapides', 'St. Charles', 'St. Landry', 'Terrebonne']

### Write a function to combine the alpha parish data

In [ ]:
'''This function takes a pandas dataframe of election results and a list of "alpha counties" and combines alpha precincts into one.
Input: election data in a pandas data frame
Optional input: list of counties/parishes that have alpha precincts
Output: Dataframe with precincts grouped by unique identifier (merges the split alphas back together)'''
def combine_alpha(df,alpha_co=alpha_co):
    df['Precinct'] = df['Precinct'].astype(str) #make sure the precinct field is a string
    #remove the ' A' by splitting the precinct field on the space for those in the alpha_co list, otherwise return the precinct column as is
    df['Precinct'] = df.apply(lambda x: x['Precinct'].split(' ')[0] if x['Parish'] in alpha_co else x['Precinct'],axis=1) 
    #join parish, ward, precinct to get a groupby field 
    df['groupby'] = df[['Parish','Ward','Precinct']].apply(lambda x: '/'.join(x),axis=1)
    #sum the votes that have the same value for group by and put in one row (this should be all that we modified to drop the ' A')
    df = df.groupby(by='groupby').sum()
    #reset the index
    df.reset_index(inplace=True)
    #add back all of the columns we got rid of except office (e.g. Parish, Ward, Precinct)
    df['Parish'] = df['groupby'].apply(lambda x: x.split('/')[0])
    df['Ward'] = df['groupby'].apply(lambda x: x.split('/')[1])
    df['Precinct'] = df['groupby'].apply(lambda x: x.split('/')[2])
    #delete groupby field, we won't need it again
    del df['groupby']
    #return the df
    return df

### Run the alpha removal function on the SoSG and SoSP files

In [ ]:
sosg = combine_alpha(SoSG)
sosp = combine_alpha(SoSP)
sosg_tot = get_column_totals(sosg)
print('sosg dictionary: ', sosg_tot)
sosp_tot = get_column_totals(sosp)
print('sosp dictionary: ', sosp_tot)
sosp_tot.update(sosg_tot)
sos_tot = sosp_tot
sosp_tot = get_column_totals(sosp)
print('sos total dictionary: ', sos_tot)
print('VEST final dictionary: ',final_tot)
matching = sos_tot == final_tot
print('Do the election totals between the processed alpha counties election data and the final VEST file match (T/F)? ', str(matching))
print('There are ', sosg.shape[0], ' precincts in the sosg file (alpha counties assigned).')
print('There are ', sosp.shape[0], ' precincts in the sosp file (alpha counties assigned).')
print('There are ', final.shape[0],' precincts in the final VEST file.')

In [ ]:
print(sosg[sosg["Parish"]=="Rapides"])
print(sosp[sosp["Parish"]=="Rapides"])

In [ ]:
'''This function retrieves a list of all parishes in a dataframe.
Input: pandas dataframe
Output: list of parishes'''
def county_list(df):
    #Get list of all unique parishes in the dataset.
    parishes = df['Parish'].unique()
    return parishes

'''This function creates a dictionary of parish names (key) and the dataframe subsetted to precincts in that parish (value). 
Input: pandas dataframe of election results
Output: Dictionary where keys are parish names and values are dataframes where the parish field value is the key.'''
def subset_county(df):
    parish_list = county_list(df)
    parish_dict = {}
    for parish in parish_list:
        parish_df = df[df.Parish == parish]
        mini_dict = {parish : parish_df}
        parish_dict.update(mini_dict)
    return parish_dict

'''This function assigns the early vote to each precinct based on respective candidate performance within a parish.
Input: pandas dataframe of election results.
Output: pandas dataframe of election results with early vote assigned to precincts within a parish, and the "Early Vote" precincts are removed.'''
def get_early_vote(df):
    county_dict = subset_county(df) #returns dictionary of each parish and the subsetted df of that parish
    not_a_vote_col = ['Office','Parish','Ward','Precinct','groupby'] #columns that are not election results
    new_parish_dfs = []
    for k,v in county_dict.items():
        early_vote_row = v[v.Ward=='Early Voting'] #subsets Early Vote ward in the parish df
        for i in v.columns:
            if i not in not_a_vote_col:
                v[i] = v[i].astype(int) #make sure the election results column is in integer
                early_vote_value = early_vote_row.iloc[0][i] #get the value of the early vote for that parish
                sum_inperson = (v[i].sum())-early_vote_value #sum all of the in-person votes in the parish
                #for each row in the parish, divide the candidate performance by the total in person vote, and then multiply that factor by the early_vote_value and add that to the original vote 
                v[i] = v[i].apply(lambda x: ((x/sum_inperson)*(early_vote_value)) + x)
        #identiy all in-person precincts
        in_person_rows = v[v.Ward != 'Early Voting']
        #reassign v(the parish df) to only the in-person precincts, now that the Early Vote as been assigned
        v = in_person_rows
        #Create a 'Code' field that we can later use for joining
        v['Parish'] = v['Parish'].apply(lambda x: x + ',')
        v['Code'] = v[['Parish','Ward','Precinct']].apply(lambda x: ' '.join(x), axis=1)
        #remove all unnecessary fields
        del v['Parish']
        del v['Ward']
        del v['Precinct']
        #add the dataframe to the list of new dataframes (one dataframe per parish)
        new_parish_dfs.append(v)
    #once the loop is done, concatonate all of the new dataframes into one dataframe
    df = pd.concat(new_parish_dfs)
    #return the df with early votes assigned and the early vote precincts removed (should reduce the overall row count by 64, as there are 64 parishes LA)
    return df

### Run the early vote functions on the sosg and sosp datasets.

In [ ]:
#Assign the early vote for each election dataset
SoSG_assigned = get_early_vote(sosg)
SoSP_assigned = get_early_vote(sosp)

#Check the row count for original and processed files
print('SoSG row count prior to assigning early vote: ', str(sosg.shape[0]))
print('SoSG row count after assigning early vote: ', str(SoSG_assigned.shape[0]))
print('SoSP row count prior to assigning early vote: ',str(sosp.shape[0]))
print('SoSP row count after assigning early vote: ',str(SoSP_assigned.shape[0]))

#Check that the correct number of rows were removed
parishes_in_la = county_list(sosg)
num_parishes = len(parishes_in_la)
diff_cases = int(sosg.shape[0])-int(SoSG_assigned.shape[0])
matching = diff_cases == num_parishes
print('Does SoSG now have ', num_parishes, ' fewer parishes than the SoSG alpha modified file (T/F)? ', str(matching))
parishes_in_la = county_list(sosp)
num_parishes = len(parishes_in_la)
diff_cases = int(sosp.shape[0])-int(SoSP_assigned.shape[0])
matching = diff_cases == num_parishes
print('Does SoSP now have ', num_parishes, ' fewer parishes than the SoSP alpha modified file (T/F)? ', str(matching))



#Check that the vote totals are still the same between the new files and the original files
SoSG_assigned_tot = get_column_totals(SoSG_assigned)
matching = SoSG_tot == SoSG_assigned_tot
print('Does SosG dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)?', str(matching))
SoSP_assigned_tot = get_column_totals(SoSP_assigned)
matching = SoSP_tot == SoSP_assigned_tot
print('Does SosP dataframe with the early vote assigned have the same vote totals for each column as the original dataset (T/F)?', str(matching))


## Look at all data and modify fields accordingly


In [ ]:
print('Precinct columns: ,', list(precincts.columns))
print('\n')
print('SoSG columns: ', list(sosg.columns))
print('\n')
print('SoSP columns: ', list(sosp.columns))
print('\n')
print('VEST columns: ', list(final.columns))
#Look here, should these print statements be changed (changed them from SoSG and SoSP)?

### Process precinct columns to match VEST

In [ ]:
precincts = precincts[['STATEFP10','COUNTYFP10','VTDST10','NAME10','geometry']]

## Join election data together

In [ ]:
election_data = pd.merge(SoSP_assigned, SoSG_assigned)

#Add Parish field back
election_data['Parish'] = election_data['Code'].apply(lambda x: x.split(',')[0])
election_data.head()

In [ ]:
#print(election_data[(election_data["Parish"]=="Lafourche")])
print(election_data[(election_data["Parish"]=="Rapides")])

In [ ]:
print(validate[validate['Final Join']=="079C4"])

## Create a 'Code' field or unique identifier for the precinct geodataframe

### Write a function to retrieve the parish name based on the parish fips

In [ ]:
'''This function uses the Census API to retrieve all of the county names and FIPS for a given state.
Input: State FIPS code as a string.
Output: pandas data frame of county fips and names for the given state.'''
def county_names(state_fips):
    """Inputs: state fips code
    Process: Retrieves a list of counties in the given state from the Census API.  
    Outputs: A list of county fips codes in the state. """
    #uses the fips input into the census api
    resp = requests.get(
        "https://api.census.gov/data/2010/dec/sf1"
        "?get=NAME&for=county:*&in=state:{}".format(state_fips)  #uses the fips input to locate the state
    )
    #retrieves the data as a json 
    test = resp.json()
    header, *rows = resp.json()
    #county column is "county"
    county_column_index = header.index("county")
    county_fips = (row[county_column_index] for row in rows) #sequence of counties 
    county_names_index = header.index("NAME")
    county_names = (row[county_names_index] for row in rows) #names of the counties
    county_fips = np.array(list(county_fips)) #make the sets into numpy arrays
    county_names = np.array(list(county_names))
    df = pd.DataFrame({'COUNTYFP10': county_fips, 'COUNTYNAMES': county_names}) #make pd dataframe of arrays
    df[['COUNTY_NAME','state']] = df.COUNTYNAMES.str.split(",",expand=True)
    del df['state']
    del df['COUNTYNAMES']
    return df  #returns the fips codes of all counties

### Run the function for Louisiana

In [ ]:
LA_counties=county_names('22')

#Create a Parish field and delete the old one
LA_counties['Parish'] = LA_counties.apply(lambda x: x['COUNTY_NAME'][:-7], axis = 1)
del LA_counties['COUNTY_NAME']

#View the retrieved dataset
LA_counties.head()
print(LA_counties)

### Merge the precinct shapefile and the retrieved dataframe

In [ ]:
precincts_m = pd.merge(precincts,LA_counties, on=['COUNTYFP10'])

#View the new dataframe
precincts_m.head()

## In order to match the election results with the precinct file, we need to perform string manipulation on the shapefile to create a common unique identifier

In [ ]:
precincts_m['Precinct1'] = precincts_m['VTDST10'].apply(lambda x: x.split('-')[1] if x.find('-')!=-1 else x )
precincts_m['PrecinctNumber'] = precincts_m['Precinct1'].str.replace('([A-Z]+)', '')
precincts_m['County'] = precincts_m['Parish']
precincts_m['PrecinctLetter'] = precincts_m['Precinct1'].str.extract('([A-Z]+)')
precincts_m['County'] = precincts_m['County'].apply(lambda x: x.replace('La Salle','Lasalle'))
precincts_m['County'] = precincts_m['County'].apply(lambda x: x.replace('the','The'))
precincts_m['PrecinctNumber'] = precincts_m['PrecinctNumber'].apply(lambda x: '{0:0>2}'.format(x))
precincts_m['PrecinctNumber'] = precincts_m.apply(lambda x: x['PrecinctNumber'].replace('0','') if (x['County']=='Catahoula') or (x['County'] == 'Grant')  or (x['County']=='Plaquemines') or (x['County']=='St. Charles') or (x['County']=='St. Helena') or (x['County']=='Winn') else x['PrecinctNumber'], axis=1)# NEW
precincts_m['PrecinctNumber'] = precincts_m.apply(lambda x: '{0:0>3}'.format(x['PrecinctNumber']) if (x['County']=='Caddo') or (x['County']=='East Baton Rouge') or (x['County'] == 'Jefferson')  or (x['County'] == 'Lafayette')  or (x['County']=='Tangipahoa') or (x['County']=='Terrebonne') else x['PrecinctNumber'], axis=1) # NEW[]
precincts_m['PrecinctNumber'] = precincts_m['PrecinctNumber'].astype(str)                                                                   
precincts_m['PrecinctLetter'] = precincts_m['PrecinctLetter'].astype(str)                                                                   
precincts_m['PrecinctLetter'] = precincts_m['PrecinctLetter'].replace('nan', '')
precincts_m['Precinct'] = precincts_m['PrecinctNumber'].str.cat(precincts_m['PrecinctLetter'],sep="")
precincts_m['Ward'] = precincts_m['VTDST10'].apply(lambda x: x.split('-')[0] if x.find('-')!=-1 else '00')
precincts_m['Ward'] = precincts_m['Ward'].apply(lambda x: '{0:0>2}'.format(x))
precincts_m['Add2Precinct'] = precincts_m['VTDST10'].apply(lambda x: x.rsplit('-',2) if x.find('-') else '0')
precincts_m['Add2Precinct'] = precincts_m['Add2Precinct'].apply(lambda x: x[2] if len(x) == 3 else '')
precincts_m['PRECINCTS'] = precincts_m[['Precinct', 'Add2Precinct']].apply(lambda x: '-'.join(x), axis=1)
precincts_m['PRECINCTS'] = precincts_m ['PRECINCTS'].apply(lambda x: x.replace('-', '') if x[-1] == '-' else x)
#precincts_m['Ward'] = precincts_m.apply(lambda x: x['Ward'].replace(x['Ward'][-1],x['PRECINCTS'][0]) if (x['County']=='Rapides') or (x['County']=='St. Tammany') or (x['County']=='Evangeline') and (x['PRECINCTS'].endswith('([A-Z]+)')) else x['Ward'], axis=1) #***
#precincts_m['Ward'] = precincts_m.apply(lambda x: x['Ward'].replace(x['Ward'][0],'0') if (x['County']=='Evangeline') else x['Ward'], axis=1)
precincts_m['Ward'] = precincts_m.apply(lambda x: x['Ward'].replace(x['Ward'],'00') if (x['County']=='Rapides') or (x['County']=='Lafayette') or (x['County']=='St. Tammany') else x['Ward'], axis=1)
precincts_m['PRECINCTS'] = precincts_m.apply(lambda x: x['PRECINCTS'].replace(x['PRECINCTS'][-1],'%temp%').replace(x['PRECINCTS'][:2],x['PRECINCTS'][-1]).replace('%temp%',x['PRECINCTS'][:2]) if (x['County']=='Rapides') or (x['County']=='St. Tammany') and (x['PRECINCTS'].endswith('([A-Z]+)')) else x['PRECINCTS'], axis=1)
precincts_m['PRECINCTS'] = precincts_m.apply(lambda x: x['PRECINCTS'].replace(x['PRECINCTS'][-1],'%temp%').replace(x['PRECINCTS'][-2],x['PRECINCTS'][-1]).replace('%temp%',x['PRECINCTS'][-2]) if (x['County']=='Rapides') or (x['County']=='St. Tammany') and (x['PRECINCTS'].startswith('([A-Z]+)')) else x['PRECINCTS'],axis=1)
#precincts_m['PRECINCTS'] = precincts_m.apply(lambda x: x['PRECINCTS'].replace(x['PRECINCTS'][-2],x['Ward'][1]) if (x['County']=='Lafayette') else x['PRECINCTS'],axis=1)
precincts_m['Ward'] = precincts_m.apply(lambda x: x['Ward'].replace(x['Ward'][1],'0') if (x['County']=='Lafayette') else x['Ward'],axis=1)
precincts_m['CountyComma'] = precincts_m['County'].apply(lambda x: x + ',')
precincts_m['Code'] = precincts_m[['CountyComma','Ward', 'PRECINCTS']].apply(lambda x: ' '.join(x), axis=1)
precincts_m['Code'] = precincts_m.apply(lambda x: x['CountyComma'] + ' 00 ' + str(x['Code'].split(' ')[1].replace('0','')) + '-' + str(x['Code'].split(' ')[2].replace('0', '')) if (x['County']=='Concordia') or (x['County']=='Morehouse') or (x['County']=='Sabine') or (x['County']=='Vermilion') else x['Code'], axis=1)
#precincts_m['Code'] = precincts_m.apply(lambda x: x['Code']+ ' A' if x['County']=='East Baton Rouge' else x['Code'], axis = 1 )##NEW***
precincts_m['Code'] = precincts_m.apply(lambda x: x['Code'].replace('-','') if (x['County'] == 'Morehouse') or (x['County']=='Vermilion') else x['Code'], axis = 1)# NEW***
#precincts_m['Code'] = precincts_m.apply(lambda x: x['Code'].replace(' A','') if (x['County']=='Ascension') else x['Code'],axis =1)##NEW***
precincts_m['Final Join'] = precincts_m[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)

#Clean the data
columns_to_drop = ['Ward','PRECINCTS','County','Precinct','Precinct1','PrecinctNumber','PrecinctLetter','Add2Precinct','PRECINCTS','CountyComma']
precincts_m  = precincts_m.drop(columns = columns_to_drop)

#Look at the output of the string matching
precincts_m.head(5000)

## Merge precincts per VEST's documentation

### Write a function to dissolve the precincts and a function to remove the old precincts

In [ ]:
'''The function takes two rows in a geodataframe and merges them into one geometry.
Inputs: Parish name (string), precinct 1 name (string), precinct 2 name (string)
Optional Inputs: rename (string), gdf (geopandas gdf), parish_col (string), vtd_col (where the precinct names are pulled from, string), and precinct_col (field that is manipulated, string)
Output: geodataframe of the dissolved precincts
'''
def precincts_for_dissolve(parish, prec1,prec2, rename='', gdf=precincts_m,parish_col = 'Parish', vtd_col = 'VTDST10', precinct_col = 'NAME10'):
    parish_gdf = gdf[(gdf[parish_col]==parish)] #subsets input df to just precincts in the input parish
    dissolve_gdf = parish_gdf[(parish_gdf[vtd_col]==prec1) | (parish_gdf[vtd_col]==prec2)] #subsets the parish df to be just the two rows that need to be merged
    vtd_list = dissolve_gdf[vtd_col] 
    if (prec1 == '6-2') or (prec1=='6-4'):
        dissolve_gdf = dissolve_gdf.dissolve(by=parish_col,aggfunc = 'last') #assigns by the last row
    else:
        dissolve_gdf = dissolve_gdf.dissolve(by=parish_col,aggfunc = 'first') #assigns by the first row
    if rename == 'rename':
        new_name = prec1.split('-')[0] #renames prec1 the first part befor the hyphen
        dissolve_gdf[vtd_col]=dissolve_gdf[vtd_col].replace(prec1, new_name)
        dissolve_gdf[precinct_col] = dissolve_gdf[precinct_col].replace('Precinct ' + prec1, 'Precinct ' + new_name) #renames the precinct name col
    return dissolve_gdf #returns a geodataframe with one row: the dissolved geometries of the two rows

'''The function takes dictionary of parishes and precincts and drops those from the geodataframe.
Input: dictionary of parishes and precincts
Option Input: parish column, vtd column
Output: geodataframe with dropped precincts'''
def drop_cases(drop_data_dict, gdf = precincts_m, parish_col = 'Parish',vtd_col = 'VTDST10'):
    proj = gdf.crs
    print('gdf shape before any alteration: ', gdf.shape)
    for k,v in drop_data_dict.items():
        parish_gdf = gdf[(gdf[parish_col]==k)]
        gdf = gdf.drop(parish_gdf.index)
        gdf.reset_index(drop=True,inplace=True)
        #print('gdf shape after dropping ', k, ' ', gdf.shape)
        #for i in v:
        #    parish_gdf = parish_gdf.drop(parish_gdf[(gdf[vtd_col] == i)].index)
        not_added = parish_gdf[parish_gdf[vtd_col].isin(v)]
        #print('removed precincts from the dataframe: ', list(not_added[vtd_col]))
        parish_gdf = parish_gdf[~parish_gdf[vtd_col].isin(v)]
        #print('precincts staying in the df form the parish: ', list(parish_gdf[vtd_col]))
        gdf = gp.GeoDataFrame(pd.concat([gdf,parish_gdf],ignore_index=True), crs = proj)
        #print('gdf shape after adding back ', k, ' and dropping merged precincts ', gdf.shape)
    return gdf

## Run the dissolve function on the precincts per VESTs documentation

***Add in the info on processing here**

In [ ]:
avoyelles = precincts_for_dissolve('Avoyelles','2-5B','6-1A')
plaquemines1 = precincts_for_dissolve('Plaquemines','2-1','2-2')
plaquemines2 = precincts_for_dissolve('Plaquemines','4-1','4-2')
plaquemines3 = precincts_for_dissolve('Plaquemines', '5-1','5-2')
st_charles1 = precincts_for_dissolve('St. Charles','2-6','2-7')
st_charles2 = precincts_for_dissolve('St. Charles','3-1','3-6')
st_charles3 = precincts_for_dissolve('St. Charles','3-3','3-4')
st_charles4 = precincts_for_dissolve('St. Charles','6-2','6-3')
st_charles5 = precincts_for_dissolve('St. Charles','6-4','6-5')

#Add all of the dissolved gdfs to a list
prec2replace = [avoyelles,plaquemines1,plaquemines2,plaquemines3,st_charles1,st_charles2,st_charles3,st_charles4,st_charles5]
#assign proj to the crs of the original file
proj = precincts_m.crs
#concatonate the geodataframes into one geodataframe
prec2replace = gp.GeoDataFrame(pd.concat(prec2replace, ignore_index=True), crs = proj)
print('There are ', prec2replace.shape[0],' precincts that are dissolved to be added back')

#create data drop dict to remove the select precincts

drop_data_dict = {'Avoyelles':['2-5B','6-1A'],
                 'Plaquemines':['2-1','2-2','4-1','4-2','5-1','5-2'],
                 'St. Charles':['2-6','2-7','3-1','3-6','3-3','3-4','6-2','6-3','6-4','6-5']}

#drop the precincts
precincts_modified = drop_cases(drop_data_dict)
print('There are ', precincts_modified.shape[0], ' precincts after dropping the cases to be removed. After adding in the merged cases there should be ', precincts_modified.shape[0]+prec2replace.shape[0])
#Add the precincts back
precincts_mod = gp.GeoDataFrame(pd.concat([precincts_modified,prec2replace],ignore_index=True), crs=proj)
print('There are ', precincts_mod.shape[0], ' after adding back the precincts.')
matching = precincts_mod.shape[0]==precincts_modified.shape[0]+prec2replace.shape[0]
print('Does the new file have the expected number of precincts (T/F)? ', str(matching))


### Modify the final shapefile so it and the precinct file are the same. 
We have to modify the 'split' precincts in the final file since we don't know what they were split on,
we will merge them back together to match the original file.

In [ ]:
#Merge the the LA counties file with the final file to get the parish names in the dataset
final_merged = pd.merge(final,LA_counties, on=['COUNTYFP10'])

#run the dissolve function for the counties that were indicated as 'split' in the metadata
final_vermilion_dis = precincts_for_dissolve('Vermilion','49B-1','49B-2')
final_wbr_dis = precincts_for_dissolve('West Baton Rouge','2A','2B','rename')
final_wbr_dis2 = precincts_for_dissolve('West Baton Rouge','11A','11B','rename')
final_p2r = [final_vermilion_dis,final_wbr_dis,final_wbr_dis2]
proj = final.crs
final_p2r = gp.GeoDataFrame(pd.concat(final_p2r,ignore_index=True), crs=proj)

#run the data drop function for the 'split' counties
drop_data_final = {'Vermilion':['49B-1','49B-2'],
                  'West Baton Rouge':['2A','2B','11A','11B']}
final_modi = drop_cases(drop_data_final,final_merged)
final_modified = gp.GeoDataFrame(pd.concat([final_modi,final_p2r],ignore_index=True),crs=proj)
final_modified['Final Join'] = final_modified[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)
print('There are ', final_modified.shape[0], ' precincts after dropping the cases to be removed. After adding in the merged cases there should be ', final_modi.shape[0]+final_p2r.shape[0])

#Add the precincts back
final_mod = gp.GeoDataFrame(pd.concat([precincts_modified,prec2replace],ignore_index=True), crs=proj)
print('There are ', final_modi.shape[0], ' after adding back the precincts.')
matching = final_mod.shape[0]==(final_modi.shape[0]+final_p2r.shape[0])
print('Does the new file have the expected number of precincts (T/F)? ', str(matching))


## Confirm that the named precincts are the same between the raw and final files

In [ ]:
#replace final file that we know is different after looking at the data

unique_final = []
for i in final_mod['Final Join']:
    unique_final.append(i)
unique_prec = []
for i in precincts_mod['Final Join']:
    unique_prec.append(i)
differing = []
for i in unique_final:
    if i not in unique_prec:
        print(i, ' not in precinct file')
        differing.append(i)
for i in unique_prec:
    if i not in unique_final:
        print(i, ' not in final file')
        differing.append(i)

print('Numer of unique values in Final Join filed in the precincts file: ', str(precincts_mod['Final Join'].nunique()))

print('There are ', len(differing), ' precincts that are different between the two files. They are ', differing)
matching = final_mod.shape[0]==precincts_mod.shape[0]
print('Are there the same number of cases in the modified VEST and precinct files (T/F)? ', str(matching))

### Verify that the geometries themselves are the same between the two files

In [ ]:
final_mod["geometry"]=final_mod.buffer(0) #buffer by 0 to remove any non polygons
precincts_mod["geometry"]=precincts_mod.buffer(0)
proj = final_mod.crs #project to same projection
precincts_mod = precincts_mod.to_crs(proj)
final_mod = final_mod.to_crs(proj)
final_mod =final_mod.sort_values(by=['Final Join']) #sort values by the unique field
precincts_mod = precincts_mod.sort_values(by=['Final Join'])
precincts_mod.reset_index(drop=True,inplace=True) #drop the index
final_mod.reset_index(drop=True,inplace=True)

#see the differencebetween the precinct and final file
file = precincts_mod.difference(final_mod)

#Difference as a percentage is very very close to 0
print('Difference between the precinct and final file area as a percentage of the precinct file area is ',str(sum(file.area)/sum(precincts_mod.area)))

#To 6 decimal places, the two files are equal 
t_f_geom = list(final_mod.geom_almost_equals(precincts_mod,decimal=6))
unique_geom = []
for i in t_f_geom:
    if i not in unique_geom:
        unique_geom.append(i)
for i in unique_geom:
    percent = str((t_f_geom.count(i)/len(t_f_geom))*100) 
    print(str(t_f_geom.count(i)) + ' precincts are ' + str(i) + ' which is ' + percent + ' percent of all precincts.')

## Join the election data to the precinct shapefile

### Subset the election data with necessary columns

In [ ]:
election_data = election_data[['G18SOSRARD', 'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO',
                  'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL', 
                   'Parish','Code']]

election_data.head()

In [ ]:
print(election_data[election_data["Code"]=="Rapides, 00 C04"])
print(election_data[election_data["Code"]=="Rapides, 00 C40"])

In [ ]:
#print(precincts_joined[precincts_joined["Code"]=="Rapides, 00 C04"])
print(precincts_joined.head())

### Since the election data totals have already been validated to the final data (before and after early vote assignment) we can join the data to the modified precincts geodataframe on the Code field

In [ ]:
#precincts_joined = pd.merge(precincts_modified,election_data,on='Code')
precincts_joined = pd.merge(precincts_modified,election_data,on=['Code'],how="inner")

print('Number of precincts in the election dataset: ', str(election_data.shape[0]))
print('Number of precincts in the modified precincts dataset: ', str(precincts_mod.shape[0]))
precincts_joined = precincts_joined.sort_values(by=['Final Join']) #sort by Final Join field
#Subset the joined field by the necessary column order
precincts_joined = precincts_joined[['STATEFP10','COUNTYFP10','VTDST10','NAME10','G18SOSRARD','G18SOSREDM','G18SOSRSTO','G18SOSRKEN','G18SOSRCRO','G18SOSRCLO','G18SOSDCOL','G18SOSDFRE','G18SOSNMOR','R18SOSRARD','R18SOSDCOL','geometry','Final Join']]
print('Number of precincts in the joined election and precincts geodataframe: ',str(precincts_joined.shape[0]))
print('Precincts that were successfully joined with election results as a percentage of election result precincts: ', str((precincts_joined.shape[0]/election_data.shape[0])*100))
print('Precincts that were successfully joined with election results as a percentage of the modified precinct geodataframe: ', str((precincts_joined.shape[0]/precincts_mod.shape[0])*100))

## Join the final VEST file with the precinct shapefile with election results

### Add final join column to VEST dataset

In [ ]:
final['Final Join'] = final[['COUNTYFP10','VTDST10']].apply(lambda x: ''.join(x),axis=1)

### Validate the precinct/election file and the VEST dataset by joining them

In [ ]:
validate = pd.merge(precincts_joined,final, on='Final Join') #merge on the 'Final Join column'

print('Number of precincts in the joined precincts dataset: ', str(precincts_joined.shape[0]))
print('Number of precincts in the final VEST dataset: ', str(final.shape[0]))
print('Number of precincts in the joined precinct and final VEST file : ',str(validate.shape[0]))
print('Precincts with election results that were successfully joined with the final file as a percentage of the precinct election results geodataframe: ', str((validate.shape[0]/precincts_joined.shape[0])*100))
print('Precincts with election results that were successfully joined with the final file as a percentage of the precincts in the final file: ', str((validate.shape[0]/final.shape[0])*100))
validate.head()

### Compare Precinct Votes in the Validated File

In [ ]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff != 0):
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]['Final Join'])
                if (diff>max_diff):
                    print("New max diff is: ", str(max_diff))
                    max_diff = diff
                    print(df.iloc[j]['Final Join'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]['Final Join'])
        else:
            matching_rows +=1
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    print("The average difference is: ", str(sum(diff_values)/len(diff_values)))

In [ ]:
print(validate[validate["Final Join"]=="0673-2"])

In [ ]:
column_List = ['G18SOSRARD', 'G18SOSREDM', 'G18SOSRSTO', 'G18SOSRKEN', 'G18SOSRCRO', 'G18SOSRCLO', 'G18SOSDCOL', 'G18SOSDFRE', 'G18SOSNMOR', 'R18SOSRARD', 'R18SOSDCOL']
validater_row(validate,column_List)

In [ ]:
print(validate[validate['Final Join']=="079C4"])

### Check the shapefiles of the merged file

### Verify the column totals of the joined file

In [ ]:
validate_tots = get_column_totals(validate)

In [ ]:
#Get all values that have a key that includes _x and add to a new list
values_x = [value for key, value in validate_tots.items() if '_x' in key]
#Remove the _x in the keys of the dict and add to a new list
keys_x = [key for key, value in validate_tots.items() if '_x' in key]

#Replace all of the _x keys with '' to remove it
new_keys_x = []
for key in keys_x:
    new_key = key.replace('_x','')
    new_keys_x.append(new_key)
    
#add the new_keys_x list and the values_x to a new dictionary of all x data
validate_x = dict(zip(new_keys_x,values_x))

#Repeate the same process as the _x data for the _y data
values_y = [value for key, value in validate_tots.items() if '_y' in key]
keys_y = [key for key, value in validate_tots.items() if '_y' in key]
new_keys_y = []
for key in keys_y:
    new_key = key.replace('_y','')
    new_keys_y.append(new_key)
validate_y = dict(zip(new_keys_y,values_y))

#print the two new dictionaries
print(validate_x)
print(validate_y)
matching = validate_x == validate_y

print('\nAre there the election totals of the succesfully joined final and precinct/election file the same (T/F)? ', str(matching))
print("\nThese differences are likely due to the unknown rounding method, which would produce inconsistencies in the totals if not looking at the *entire* total (which aren't since we didn't have a 100% successful join.)")

#Determine the difference between the _x and the _y vote totals as a percentage (absolute value)
vote_per_list = []
for k,v in validate_x.items():
    for k2,v2 in validate_y.items():
        if k==k2: #if the keys are the same
            print('Difference in vote between precinct/elections file and final VEST file for ', k, ' is ', str(abs(v2-v)))
            vote_per = abs(((v2-v)/v2)*100)
            print('Precentage of vote that is missing: ', str(vote_per))
            vote_per_list.append(vote_per)
            

print('\nThe maximum percentage difference in vote after joining is: ', max(vote_per_list))
print('\nThe minimum percentage difference in vote after joining is: ', min(vote_per_list))

## Recommendations for Report

Raw data available?	Yes

Processing steps available?	Yes

Able to replicate joining election data and shapefiles?	Yes, but this is slightly less good one but given the rate, but I would say yes still. You can print out the difference from like an outer join to see the ones that are off and you can tell that the vast majority are due to string matching not being 100% perfect.

Able to replicate by joining demographic data?	N/A

Able to replicate by joining boundary data?	N/A

Successfully ran validation? Yes